In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from time import time
import datetime as dt
import importlib

import aux_funcs as aux

In [2]:
test = aux.read_test_data()

test.csv read in 35.0 s
Mem. usage decreased to 596.49 Mb (53.1% reduction)
(41697600, 10)
   row_id  building_id  meter  timestamp  hour  day  year  weeknumber  \
0       0            0      0 2017-01-01     0    1  2017          52   
1       1            1      0 2017-01-01     0    1  2017          52   
2       2            2      0 2017-01-01     0    1  2017          52   
3       3            3      0 2017-01-01     0    1  2017          52   
4       4            4      0 2017-01-01     0    1  2017          52   

   weekday  month  
0        6      1  
1        6      1  
2        6      1  
3        6      1  
4        6      1  


In [3]:
weather_full_test = aux.read_full_weather_test_data()

Weather_test_fulldata.csv read in 0.3 s
Mem. usage decreased to  2.94 Mb (54.2% reduction)


In [5]:
building = aux.read_building_data()

building_metadata.csv read in 0.2 s
Mem. usage decreased to  0.03 Mb (60.3% reduction)
(1449, 7)
   site_id  building_id primary_use  square_feet  year_built  floor_count  \
0        0            0   Education         7432      2008.0          NaN   
1        0            1   Education         2720      2004.0          NaN   
2        0            2   Education         5376      1991.0          NaN   
3        0            3   Education        23685      2002.0          NaN   
4        0            4   Education       116607      1975.0          NaN   

   log_square_feet  
0         8.913550  
1         7.908387  
2         8.589700  
3        10.072597  
4        11.666565  


In [6]:
building.drop(columns=['year_built','floor_count'],inplace=True)
building.head()

,site_id,building_id,primary_use,square_feet,log_square_feet
0,0,0,Education,7432,8.913550
1,0,1,Education,2720,7.908387
2,0,2,Education,5376,8.589700
3,0,3,Education,23685,10.072597
4,0,4,Education,116607,11.666565


### join

In [7]:
#merge tables
testbw = test.join(building, on='building_id', rsuffix = 'r')
testbw.drop(columns=['building_idr'],inplace=True)
print(testbw.shape)
testbw.head()

(41697600, 14)


,row_id,building_id,meter,timestamp,hour,day,year,weeknumber,weekday,month,site_id,primary_use,square_feet,log_square_feet
0,0,0,0,2017-01-01,0,1,2017,52,6,1,0,Education,7432,8.913550
1,1,1,0,2017-01-01,0,1,2017,52,6,1,0,Education,2720,7.908387
2,2,2,0,2017-01-01,0,1,2017,52,6,1,0,Education,5376,8.589700
3,3,3,0,2017-01-01,0,1,2017,52,6,1,0,Education,23685,10.072597
4,4,4,0,2017-01-01,0,1,2017,52,6,1,0,Education,116607,11.666565


In [8]:
testbw = pd.merge(testbw, weather_full_test,left_on=['site_id','timestamp'],right_on=['site_id','timestamp'],how='left')
print(testbw.shape)
testbw.head()

(41697600, 15)


,row_id,building_id,meter,timestamp,hour,day,year,weeknumber,weekday,month,site_id,primary_use,square_feet,log_square_feet,air_temperature
0,0,0,0,2017-01-01,0,1,2017,52,6,1,0,Education,7432,8.913550,17.796875
1,1,1,0,2017-01-01,0,1,2017,52,6,1,0,Education,2720,7.908387,17.796875
2,2,2,0,2017-01-01,0,1,2017,52,6,1,0,Education,5376,8.589700,17.796875
3,3,3,0,2017-01-01,0,1,2017,52,6,1,0,Education,23685,10.072597,17.796875
4,4,4,0,2017-01-01,0,1,2017,52,6,1,0,Education,116607,11.666565,17.796875


In [ ]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor

predictor_file = 'adaboost_01.pkl'
# read trees
import pickle
with open(predictor_file,'rb') as file:
    dtrs = pickle.load(file)

In [46]:
# initialise with nans
testbw['log_meter_reading'] = np.nan

In [9]:
features_list=['weeknumber','weekday','hour','log_square_feet','air_temperature']

In [10]:
subgroups = testbw.groupby(['primary_use','meter'])['meter'].count()
subgroups

primary_use                    meter
Education                      0        9408240
                               1        3784320
                               2        2382720
                               3        1226400
Entertainment/public assembly  0        3136080
                               1         770880
                               2         525600
                               3         280320
Food sales and service         0          87600
                               1          70080
                               2          35040
                               3          35040
Healthcare                     0         367920
                               1         227760
                               2         157680
                               3          52560
Lodging/residential            0        2540400
                               1         981120
                               2         788400
                               3         192720
Man

In [28]:
def section_predict(dtr, testbw, cond, features_list, thr=100000, out_column='log_meter_reading',verbose=0):
    # breaks down prediction
    #
    # dtr, regression model
    # testbw, dataframe with data
    # cond, vectors of booleans
    # features_list, features to be used for predict
    # thr, threshold where to break down prediction

    #indices where cond is true
    print('Extracting sub-dataframe...')
    ixcond = testbw.loc[cond].index
    
    if len(ixcond) < thr:
        # if below threshold, do it all at once
        testbw.loc[cond,out_column] =\
            dtr.predict(testbw.loc[cond, features_list])
    else:
        # bin definition
        bins = np.arange(0,len(ixcond),thr)
        print('Predicting with {:d} sections'.format(len(bins)))
        
        # loop
        for k in range(len(bins)-1):
            low = bins[k]
            high = bins[k+1]
            if verbose>0:
                print('Section {:d} out of {:d}'.format(k, len(bins)))
            # predict
            testbw.loc[ixcond[low:high],out_column] = dtr.predict(testbw.loc[ixcond[low:high], features_list])
                      
        # last bit
        ixend = bins[-1]
        testbw.loc[ixcond[ixend:],out_column] =\
            dtr.predict(testbw.loc[ixcond[ixend:], features_list])
                      
    return testbw
        

In [18]:
# delete vars to free memory (hopefully)
del weather_full_test
del building
del test

In [ ]:
subix = subgroups.index
## CHANGE INDEX TO 0 IN RANGE!!!!!
for k in range(0, len(subgroups)):

    use = subix[k][0]
    meter = subix[k][1]
    dtr = dtrs[k]
    
    print('----------------')
    print(k, use, meter)
    print('   Filter by use and meter...')
    # condition
    cond = (testbw['primary_use']== use) & (testbw['meter']==meter)
    # predict
    t0 = time()
    print('   Predict...')
    testbw = section_predict(dtr, testbw, cond, features_list, verbose=1)
    print('   Prediction in {:.1f}'.format(time()-t0))
    print('   Done!')
    print('')

----------------
0 Education 0
   Filter by use and meter...
   Predict...
Extracting sub-dataframe...
Predicting with 95 sections
Section 0 out of 95
Section 1 out of 95
Section 2 out of 95
Section 3 out of 95
Section 4 out of 95
Section 5 out of 95
Section 6 out of 95
Section 7 out of 95
Section 8 out of 95
Section 9 out of 95
Section 10 out of 95
Section 11 out of 95
Section 12 out of 95
Section 13 out of 95
Section 14 out of 95
Section 15 out of 95
Section 16 out of 95
Section 17 out of 95
Section 18 out of 95
Section 19 out of 95
Section 20 out of 95
Section 21 out of 95
Section 22 out of 95
Section 23 out of 95
Section 24 out of 95
Section 25 out of 95
Section 26 out of 95
Section 27 out of 95
Section 28 out of 95
Section 29 out of 95
Section 30 out of 95
Section 31 out of 95
Section 32 out of 95
Section 33 out of 95
Section 34 out of 95
Section 35 out of 95
Section 36 out of 95
Section 37 out of 95
Section 38 out of 95
Section 39 out of 95
Section 40 out of 95
Section 41 out of 

Predicting with 8 sections
Section 0 out of 8
Section 1 out of 8
Section 2 out of 8
Section 3 out of 8
Section 4 out of 8
Section 5 out of 8
Section 6 out of 8
   Prediction in 22.9
   Done!

----------------
19 Lodging/residential 3
   Filter by use and meter...
   Predict...
Extracting sub-dataframe...
Predicting with 2 sections
Section 0 out of 2
   Prediction in 5.7
   Done!

----------------
20 Manufacturing/industrial 0
   Filter by use and meter...
   Predict...
Extracting sub-dataframe...
Predicting with 2 sections
Section 0 out of 2
   Prediction in 7.0
   Done!

----------------
21 Manufacturing/industrial 1
   Filter by use and meter...
   Predict...
Extracting sub-dataframe...
   Prediction in 2.2
   Done!

----------------
22 Manufacturing/industrial 2
   Filter by use and meter...
   Predict...
Extracting sub-dataframe...
   Prediction in 1.5
   Done!

----------------
23 Office 0
   Filter by use and meter...
   Predict...
Extracting sub-dataframe...
Predicting with 48 s

In [ ]:
# convert from log to linear
testbw['meter_reading'] = np.exp(testbw['log_meter_reading'])-1

In [ ]:
testbw.head()

In [ ]:
#export results
filename = 'submission_03_191028.csv'
# 
testbw.loc[:,['row_id','meter_reading']].to_csv(filename,index=False)

In [ ]:
aux.get_lines(filename)